In [1]:
!ls /kaggle/input/notebooks/kagglertw/shemagh-simple-ddq-map-50-o-inference6

class1_only_127.png  comparison_127.png  custom.css
class1_only_224.png  comparison_224.png  __huggingface_repos__.json
class1_only_327.png  comparison_327.png  mmdetection
class1_only_40.png   comparison_40.png	 __notebook__.ipynb
class1_only_419.png  comparison_419.png  __output__.json
class1_only_476.png  comparison_476.png  __results___files
class1_only_535.png  comparison_535.png  __results__.html
class1_only_720.png  comparison_720.png  submission.csv
class1_only_746.png  comparison_746.png  updated_predictions.csv
class1_only_826.png  comparison_826.png


In [2]:
import pandas as pd

In [3]:
sub = pd.read_csv('/kaggle/input/notebooks/kagglertw/shemagh-simple-ddq-map-50-o-inference6/updated_predictions.csv')

In [4]:
sub

,filename,right_place,prediction_string,has_class_1
0,0.jpg,0,0 0.894389 0.428531 0.109503 0.208710 0.200126,0
1,1.jpg,0,1 0.905228 0.351301 0.500090 0.525243 0.958017,1
2,10.jpg,0,-,0
3,100.jpg,0,1 0.905953 0.500398 0.549735 0.998941 0.900081,1
4,101.jpg,0,1 0.904297 0.263718 0.644881 0.527239 0.708128,1
...,...,...,...,...
837,95.jpg,0,1 0.903652 0.501548 0.551440 0.993263 0.895085,1
838,96.jpg,0,-,0
839,97.jpg,0,-,0
840,98.jpg,0,-,0


In [5]:
sub = sub[sub['right_place'] == 1]

In [ ]:
import pandas as pd
import numpy as np

def process_predictions(df):
    """
    For each row containing both class 0 and class 1:
    - Find the closest class 0 box to each class 1 box
    - Check if class 1's left edge comes after class 0's left edge (visually to the right)
    - Check if class 1's bottom edge is above class 0's bottom edge (class 1 is higher)
    """
    results = []
    
    for idx, row in df.iterrows():
        filename = row['filename']
        pred_string = row['prediction_string']
        
        
        if pred_string == '-':
            continue
        
        
        predictions = pred_string.strip().split()
        
        
        boxes = []
        for i in range(0, len(predictions), 6):
            if i + 5 < len(predictions):
                cls = int(predictions[i])
                score = float(predictions[i + 1])
                x = float(predictions[i + 2])
                y = float(predictions[i + 3])
                w = float(predictions[i + 4])
                h = float(predictions[i + 5])
                boxes.append({
                    'class': cls,
                    'score': score,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h,
                    'left': x - w/2,  
                    'right': x + w/2,
                    'top': y - h/2,
                    'bottom': y + h/2  
                })
        
        
        class_0_boxes = [b for b in boxes if b['class'] == 0]
        class_1_boxes = [b for b in boxes if b['class'] == 1]
        
        
        if not class_0_boxes or not class_1_boxes:
            continue
        
        
        for c1_box in class_1_boxes:
            min_dist = float('inf')
            closest_c0_box = None
            closest_c0_score = None
            
            
            for c0_box in class_0_boxes:
                dist = np.sqrt((c1_box['x'] - c0_box['x'])**2 + 
                              (c1_box['y'] - c0_box['y'])**2)
                if dist < min_dist:
                    min_dist = dist
                    closest_c0_box = c0_box
                    closest_c0_score = c0_box['score']
            
            
            if closest_c0_box:
                c1_left = c1_box['left']
                c0_left = closest_c0_box['left']
                c1_bottom = c1_box['bottom']
                c0_bottom = closest_c0_box['bottom']
                
                
                comes_after = c1_left > c0_left
                
                
                is_above = c1_bottom < c0_bottom
                
                
                both_conditions_met = comes_after and is_above
                
                results.append({
                    'filename': filename,
                    'right_place': row['right_place'],
                    'class_0_score': closest_c0_score,
                    'class_0_left': c0_left,
                    'class_0_bottom': c0_bottom,
                    'class_1_left': c1_left,
                    'class_1_bottom': c1_bottom,
                    'class_1_comes_after': comes_after,
                    'class_1_is_above': is_above,
                    'both_conditions_met': both_conditions_met
                })
    
    return pd.DataFrame(results)

result_df = process_predictions(sub)

filtered_results = result_df[result_df['both_conditions_met'] == True]

In [7]:
filtered_results['filename'].tolist()

['794.jpg']

In [8]:
sub = pd.read_csv('/kaggle/input/notebooks/kagglertw/shemagh-simple-ddq-map-50-o-inference6/updated_predictions.csv')

In [9]:
for image_id in filtered_results['filename'].tolist():
    sub.loc[sub['filename'] == image_id, 'right_place'] = 0

sub[['filename', 'right_place', 'prediction_string']].to_csv('submission.csv', index=False)